# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#import all neccessary libraries to perform the analysis for the assignment.
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Using the panda's read_csv () to retrieve the data for analysis
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
# Since the column "homeowner" is a categorical data, so we need to convert it to numerical data for ease of analysis.  We use get_dummies() to convert the column into 3 separate columns 
# "homeowner_mortage", "homeowner_own", "homeowner_rent" with numerical data
df = pd.get_dummies(df, columns= ["homeowner"])
df.head()



,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,low_risk,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,low_risk,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,low_risk,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,low_risk,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,low_risk,1,0,0


# Split the Data into Training and Testing

In [5]:
# Define new variable X and create the features by dropping the column "loan_status";
X = df.copy()
X = df.drop(columns="loan_status")

#  create new variable y and to create and to store our target
y = df ["loan_status"]

In [6]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [7]:
# Using the value_counts() function to check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [8]:
# Splitting the data into training and testing sets by creating X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=78,
                                                    stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# To avoid distortion in the analysis with raw data, we would scale the data with sklearn library's StandardScaler and to create an instance
from sklearn.preprocessing import StandardScaler
# Define a new variable named "scaler" as the StandScaler function
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset 
X_scaler = scaler.fit(X_train)


In [11]:
# Scale the training and testing data
scale_x_train = X_scaler.transform(X_train)

scale_x_test = X_scaler.transform(X_test)
scale_x_test

array([[ 3.78390456e-01,  3.94212360e-01,  3.90439641e-01, ...,
         1.00337618e+00, -8.12267514e-01, -3.41012879e-01],
       [-2.90095244e-01, -2.90091147e-01, -2.90085745e-01, ...,
        -9.96635184e-01,  1.23112150e+00, -3.41012879e-01],
       [-3.60137261e-03, -1.89168800e-04, -3.54874017e-03, ...,
        -9.96635184e-01,  1.23112150e+00, -3.41012879e-01],
       ...,
       [ 4.41476060e-02,  3.01494103e-02,  3.22683854e-02, ...,
         1.00337618e+00, -8.12267514e-01, -3.41012879e-01],
       [ 4.26139435e-01,  4.32416496e-01,  4.38195808e-01, ...,
         1.00337618e+00, -8.12267514e-01, -3.41012879e-01],
       [ 3.78390456e-01,  3.89717755e-01,  3.90439641e-01, ...,
        -9.96635184e-01,  1.23112150e+00, -3.41012879e-01]])

# Simple Logistic Regression

In [12]:
# To perform a simple logistic regression, we import logistic Regression model to 
# carry the analysis.  Logistic Regression was carried out and store in the model variable
# The model is fitted with scaled_x_train and y_train data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(scale_x_train, y_train)

LogisticRegression(random_state=1)

In [13]:
# From SKlearn to import balanced_accuracy score and
# to calculate the balanced accuracy score using the actual and predicted y values in the model

from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(scale_x_test)
balanced_accuracy_score(y_test, y_pred)

0.9901346127192281

In [14]:
# From SKlearn to import confusion matrix and to display the confusion matrix using the actual and predicted y values in the model
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  616,     9],
       [  100, 18659]], dtype=int64)

In [15]:
# From SKlearn to import inbalanced classification report and to print the imbalanced classification report using the actual and predicted y values in the model
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.98       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.98     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [16]:
# From imblearn library to import the RandomOver sampler to resample the training data (X_trained; Y_trained); random state will kept at 1 for comparison
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)

# To fit the resampling, X_train, Y_train data is used and output the ROS X resampled, and ROS Y resampled
ros_X_resampled,ros_y_resampled = ros.fit_resample (X_train, y_train)


# Based on the results of RandomOvr Sampler to view the count of target classes with Counter
Counter(ros_y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [17]:
# To perform a simple logistic regression, we import logistic Regression model to 
# carry the analysis.  Logistic Regression was carried out and store in the random_over_model variable
# The random_over_model is fitted with ros_x_resampled and ros_y_resampled data
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

random_over_model = LogisticRegression (solver="lbfgs", random_state=1)
random_over_model.fit(ros_X_resampled, ros_y_resampled)

LogisticRegression(random_state=1)

In [18]:
# From SKlearn to import balanced_accuracy score and
# to calculate the balanced accuracy score using the actual and predicted ROS_y_pred values in the model

ros_y_pred = random_over_model.predict(X_test)
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test,ros_y_pred)

0.9956279972279972

In [19]:
# From SKlearn to import confusion matrix and
# to calculate the confusion matrix using the actual and predicted ROS_y_pred values in the model

from sklearn.metrics import confusion_matrix
confusion_matrix (y_test,ros_y_pred)

array([[  623,     2],
       [  104, 18655]], dtype=int64)

In [20]:
# From SKlearn to import imbalanced classification report and to  
# print the imbalanced classification report based on the actual and predicted ROS_y_pred values in the model
from imblearn.metrics import classification_report_imbalanced
print (classification_report_imbalanced(y_test,ros_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      1.00      0.99       625
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18759

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



### SMOTE Oversampling

In [21]:
# From imblearn library to import the SMOTE to resample the training data (X_trained; Y_trained); random state will kept at 1 for comparison
from imblearn.over_sampling import SMOTE
X_smote_resampled, y_smote_resampled = SMOTE (random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train)

from collections import Counter


# Based on the SMOTE resampling results to view the count of target classes with Counter
Counter (y_smote_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [22]:
# The SMOTE model is fitted with X_smote_resampled and y_smote_resampled data
# Train the model using the resampled data 
smote_model = LogisticRegression (solver ="lbfgs", random_state=1, max_iter=2000)
smote_model.fit(X_smote_resampled, y_smote_resampled)

LogisticRegression(max_iter=2000, random_state=1)

In [23]:
# To make prediction based on the SMOTE model with X_test data and to calculate the balanced accuracy score of the model

smote_y_pred = smote_model.predict(X_test)
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test,smote_y_pred)

0.9953881123727277

In [24]:
# From SK learn library to import confusion matrix to display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix (y_test,smote_y_pred)

array([[  623,     2],
       [  113, 18646]], dtype=int64)

In [25]:
# From SK learn library to import imbalanced classification to print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print (classification_report_imbalanced(y_test,smote_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      1.00      0.99       625
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18759

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [26]:
#  From imblearn library to import the ClsuterCentroids to resample the training data (X_trained; Y_trained) by undersampling method; random state will kept at 1 for comparison
#  Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
Cluster_X_resampled, Cluster_y_resampled = ClusterCentroids(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train)

from collections import Counter


# Based on the ClusterCentroids method of resampling to view the count of target classes with Counter
Counter(Cluster_y_resampled)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [27]:
# To perform a simple logistic regression, we import logistic Regression model to 
# carry the analysis.  Train the regression model using the resampled data from ClusterCentroids' results (Cluster_X_resampled, Cluster_y_resampled)

from sklearn.linear_model import LogisticRegression
Cluster_model = LogisticRegression(solver='lbfgs', random_state=1)
Cluster_model.fit(Cluster_X_resampled, Cluster_y_resampled)

LogisticRegression(random_state=1)

In [28]:
# To make prediction based on the above regression model with X_test data and to calculate the balanced accuracy score of the model
from sklearn.metrics import balanced_accuracy_score
Cluster_y_pred = Cluster_model.predict(X_test)
balanced_accuracy_score(y_test, Cluster_y_pred)

0.9885346127192282

In [29]:
#From SK learn library to import confusion matrix to display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, Cluster_y_pred)

array([[  614,    11],
       [  100, 18659]], dtype=int64)

In [30]:
# From SK learn library to import classification_report_imbalanced to print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, Cluster_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.98      0.99      0.92      0.99      0.98       625
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
#  From imblearn library to import the SMOTEENN algorithm for resampling the training data (X_trained; Y_trained) by the combination method; random state will kept at 1 for comparison

# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN (random_state=1, sampling_strategy=1.0)

sm_X_resampled, sm_y_resampled = sm.fit_resample(X_train, y_train)


# View the count of target classes with Counter
Counter (sm_y_resampled)

Counter({'high_risk': 55605, 'low_risk': 55905})

In [32]:
# Train the Logistic Regression model using the resampled data from SMOTEENNs' results (sm_X_resampled, sm_y_resampled) to perform logistic Regression
from sklearn.linear_model import LogisticRegression

sm_model = LogisticRegression(solver ='lbfgs', random_state=1)
sm_model.fit (sm_X_resampled, sm_y_resampled)

LogisticRegression(random_state=1)

In [33]:
# To make prediction based on the above regression model with X_test data and to calculate the balanced accuracy score of the model
from sklearn.metrics import balanced_accuracy_score
sm_y_pred = sm_model.predict(X_test)
balanced_accuracy_score(y_test,sm_y_pred)

0.9954414201183432

In [34]:
# From SK learn library to import confusion matrix to display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix (y_test, sm_y_pred)

array([[  623,     2],
       [  111, 18648]], dtype=int64)

In [35]:
# From SK learn library to import classification_report_imbalanced to print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print (classification_report_imbalanced(y_test,sm_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      1.00      0.99       625
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18759

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   Naive Random Oversampling model has the best balanced accuracy score of 0.995628. 

2. Which model had the best recall score?

    Models of Naive Random Oversampling, SMOTE and Combination have the best recall score of 1.00.

3. Which model had the best geometric mean score?
   
    Models of Naive Random Oversampling, SMOTE and Combination have the best geometric mean score of 1.00.
